In [2]:
import requests
import json
import pandas as pd

In [3]:
res = requests.get("https://cdn-api.co-vin.in/api/v2/admin/location/states")

In [4]:
states = pd.DataFrame(json.loads(res.text)["states"])

In [5]:
states.to_csv("../data/states.csv", index = False) #sates with id 

In [6]:
df_states = pd.read_csv("../data/states.csv")

In [7]:
df_states.head(5)

,state_id,state_name
0,1,Andaman and Nicobar Islands
1,2,Andhra Pradesh
2,3,Arunachal Pradesh
3,4,Assam
4,5,Bihar


In [8]:
df_district_all = None
for state_id in df_states["state_id"].values.tolist():
    state_name = df_states.loc[df_states['state_id'] == state_id, "state_name"].item()
    res = requests.get("https://cdn-api.co-vin.in/api/v2/admin/location/districts/{}".format(state_id))
    df_district = pd.DataFrame(json.loads(res.text)["districts"])
    df_district["state_name"] = state_name
    if df_district_all is None:
        df_district_all = df_district
    else:
        df_district_all = pd.concat([df_district_all, df_district], ignore_index=True)

df_district_all = df_district_all.sort_values("district_name")
df_district_all.to_csv("../data/districts.csv", index=False)    #District_id

In [9]:
df_district_all

,district_id,district_name,state_name
598,582,Adilabad,Telangana
326,320,Agar,Madhya Pradesh
324,796,Agatti Island,Lakshadweep
639,622,Agra,Uttar Pradesh
158,154,Ahmedabad,Gujarat
...,...,...,...
307,285,Yadgir,Karnataka
220,197,Yamunanagar,Haryana
492,478,Yanam,Puducherry
411,368,Yavatmal,Maharashtra


## Search by district id 

In [74]:
#Example

district_id = "240" #Ranchi 
date_from = "01-05-2021"

In [75]:
URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={}&date={}" \
      .format(district_id,date_from)
res = requests.get(URL)

In [76]:
calender_df = pd.DataFrame(json.loads(res.text)["centers"])

In [77]:
calender_df.head(5)

,center_id,name,state_name,district_name,block_name,pincode,lat,long,from,to,fee_type,sessions,vaccine_fees
0,583628,Resaldar Baba Doranda,Jharkhand,Ranchi,Ranchi Urban,834001,23,84,09:00:00,17:00:00,Free,[{'session_id': '3dd65f93-2b54-4fc2-b4ac-5931f...,NaN
1,607446,SEVA SADAN,Jharkhand,Ranchi,Ranchi Urban,834001,0,0,10:00:00,18:00:00,Paid,[{'session_id': '32acb0e5-bf21-4998-84d3-32483...,NaN
2,630703,TAMAR PANCHAYAT,Jharkhand,Ranchi,Tamar,835225,23,85,09:00:00,17:00:00,Free,[{'session_id': '512b0f90-b0a5-489d-b661-ee736...,NaN
3,672725,GURUDWARA PISKA MORE,Jharkhand,Ranchi,Ranchi Urban,834005,23,85,09:00:00,17:00:00,Free,[{'session_id': 'd1592630-b864-4785-ae39-3f74f...,NaN
4,563481,NAMKUM PANCHYAT,Jharkhand,Ranchi,Namkum,834010,0,0,09:00:00,17:00:00,Free,[{'session_id': '1f8956a3-bb1f-4d87-9b5e-31546...,NaN


In [78]:
calender_df.dtypes

center_id         int64
name             object
state_name       object
district_name    object
block_name       object
pincode           int64
lat               int64
long              int64
from             object
to               object
fee_type         object
sessions         object
vaccine_fees     object
dtype: object

In [80]:
calender_df_new = calender_df[calender_df["pincode"] == 834002]
calender_df_new

,center_id,name,state_name,district_name,block_name,pincode,lat,long,from,to,fee_type,sessions,vaccine_fees
16,609966,HARMU HOSPITAL,Jharkhand,Ranchi,Ranchi Urban,834002,0,0,09:00:00,14:30:00,Paid,[{'session_id': 'f58e56c6-8096-4ee4-bba7-da258...,NaN
19,683331,GOVT MIDDLE SCHOOL DHOBI GHAT,Jharkhand,Ranchi,Ranchi Urban,834002,23,85,09:00:00,17:00:00,Free,[{'session_id': '5e638bb6-d89a-420c-9fad-07f59...,NaN
20,630437,RESALDAR 1,Jharkhand,Ranchi,Ranchi Urban,834002,23,85,09:00:00,18:00:00,Free,[{'session_id': '7b219df3-44a5-4622-b317-e678b...,NaN
23,616428,COMMUNITY CENTRE ASHOKE NAGER,Jharkhand,Ranchi,Ranchi Urban,834002,23,85,09:00:00,17:00:00,Free,[{'session_id': '3d09ae7f-3365-4267-b79b-0e7ac...,NaN
33,672724,NEPAL HOUSE,Jharkhand,Ranchi,Ranchi Urban,834002,23,85,09:00:00,17:00:00,Free,[{'session_id': '900e8b4a-9ad3-4a42-8ca3-18f29...,NaN


In [15]:
calender_df = calender_df[['name', 'state_name', 'district_name',
       'pincode', 'fee_type','vaccine_fees']]

In [16]:
calender_df.columns

Index(['name', 'state_name', 'district_name', 'pincode', 'fee_type',
       'vaccine_fees'],
      dtype='object')

In [17]:
calender_df.to_csv("../data/vaccine_deatils_7days.csv", index=False)

In [19]:
calender_df

,name,state_name,district_name,pincode,fee_type,vaccine_fees
0,Resaldar Baba Doranda,Jharkhand,Ranchi,834001,Free,NaN
1,SEVA SADAN,Jharkhand,Ranchi,834001,Paid,NaN
2,TAMAR PANCHAYAT,Jharkhand,Ranchi,835225,Free,NaN
3,GURUDWARA PISKA MORE,Jharkhand,Ranchi,834005,Free,NaN
4,NAMKUM PANCHYAT,Jharkhand,Ranchi,834010,Free,NaN
5,DSP HQ,Jharkhand,Ranchi,834001,Free,NaN
6,BOOTI GOVT SCHOOL,Jharkhand,Ranchi,834001,Free,NaN
7,CHANHO PANCHYAT,Jharkhand,Ranchi,835214,Free,NaN
8,BURMU CHC,Jharkhand,Ranchi,835214,Free,NaN
9,ORMANJHI CHC,Jharkhand,Ranchi,835219,Free,NaN
